<a href="https://colab.research.google.com/github/Vardhishna/CNN_model/blob/main/CNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 5.3MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78937 sha256=8af585c2c21da9f7999377d3a3c2edf4b2eb5d50d49bd00d3da0adabb2cc18cd
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=e5d09427cb75fc58f7f83df13c7e1b570989f74ef9465691d2b1a334ee95e258
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [3]:
import tensorflow as tf
from tensorflow import keras 
import numpy as np

In [4]:
fm = keras.datasets.fashion_mnist

In [5]:
(train_image, train_labels), (test_images, test_labels)= fm.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
train_image = train_image/255.0
test_images = test_images/255.0

In [7]:
train_image.shape

(60000, 28, 28)

In [8]:
train_image = train_image.reshape(len(train_image), 28,28,1)
test_images = test_images.reshape(len(test_images), 28,28,1)

In [9]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model


In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [11]:
search = RandomSearch(build_model, objective= 'val_accuracy',
                      max_trials= 5, 
                      directory= 'output', 
                      project_name= 'fashion')

In [12]:
search.search(train_image, train_labels, epochs=3, validation_split= 0.2)

Trial 5 Complete [00h 00m 14s]
val_accuracy: 0.9097499847412109

Best val_accuracy So Far: 0.9097499847412109
Total elapsed time: 00h 01m 20s
INFO:tensorflow:Oracle triggered exit


In [13]:
model = search.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 48)        480       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        20784     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 80)                2211920   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 2,233,994
Trainable params: 2,233,994
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(train_image, train_labels, epochs= 10, validation_split= 0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.1508 - accuracy: 0.9453 - val_loss: 0.2550 - val_accuracy: 0.9127
Epoch 5/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.1023 - accuracy: 0.9624 - val_loss: 0.3206 - val_accuracy: 0.9083
Epoch 6/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0674 - accuracy: 0.9750 - val_loss: 0.3530 - val_accuracy: 0.9168
Epoch 7/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0486 - accuracy: 0.9821 - val_loss: 0.3717 - val_accuracy: 0.9122
Epoch 8/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0358 - accuracy: 0.9866 - val_loss: 0.4246 - val_accuracy: 0.9070
Epoch 9/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0266 - accuracy: 0.9904 - val_loss: 0.4385 - val_accuracy: 0.9158
Epoch 10/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0230 - accuracy: 0.9916 - val_loss: 0.5371 - val_accuracy